In [1]:
import torch
from ultralytics import YOLO
import cv2
import time
import json
import numpy as np
import os

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a model
model = YOLO('yolov8x.pt')  # pretrained YOLOv8x model
source = r"C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Data"
annotation_path = r"C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Annotation"

# Define a function to calculate the Intersection over Union (IoU) of two bounding boxes
def calculate_iou(box1, box2): 
    x1, y1, x2, y2 = box1 #YOLO
    x1_gt, y1_gt, x2_gt, y2_gt = box2 #Json

    # Calculate the intersection area
    x1_i = max(x1, x1_gt)
    y1_i = max(y1, y1_gt)
    x2_i = min(x2, x2_gt)
    y2_i = min(y2, y2_gt)
    intersection_area = max(0, x2_i - x1_i) * max(0, y2_i - y1_i)

    # Calculate the union area
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_gt - x1_gt) * (y2_gt - y1_gt)
    union_area = box1_area + box2_area - intersection_area

    # Calculate the IoU
    iou = intersection_area / union_area  #if union_area != 0 else 1

    return iou

# Initialize a list to store the IoU scores for each result
iou_scores = []

# Get the list of image files
image_files = os.listdir(source)

cv2.namedWindow('scene', cv2.WINDOW_NORMAL)
cv2.resizeWindow('scene',1200, 1200)

# Process each image file
for image_file in image_files:
    # Run inference on the image
    results = model(os.path.join(source, image_file))

    for result in results: #to remove this loop
        boxes = result.boxes  # Boxes object for bounding box outputs

        # Load the ground truth from the corresponding JSON file
        json_file = os.path.join(annotation_path, os.path.splitext(image_file)[0] + '.json')
        with open(json_file) as f:
            ground_truth = json.load(f)

        jason_bboxes = ground_truth['annotations'][0]['bbox']

        # Convert the bounding box to two points
        x1y1 = (int(jason_bboxes[0]), int(jason_bboxes[1]))
        x2y2 = (int(jason_bboxes[0] + jason_bboxes[2]), int(jason_bboxes[1] + jason_bboxes[3]))

        # Convert the image from BGR to RGB
        img_rgb = cv2.cvtColor(result.orig_img, cv2.COLOR_BGR2RGB)


        # Draw bounding boxes on the image
        for box in boxes.xyxy:
            cv2.rectangle(img_rgb, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            cv2.rectangle(img_rgb, (x1y1[0], x1y1[1]) , (x2y2[0], x2y2[1]), (255, 255, 0), 2)

            # For each detected box, calculate the IoU with the ground truth box
            iou = calculate_iou(box, (x1y1[0], x1y1[1] , x2y2[0], x2y2[1]))
            iou_scores.append(iou)

        # Display the image
        cv2.imshow('scene', img_rgb)

        # Wait for 0.5 seconds
        cv2.waitKey(10)

# Close the image window
cv2.destroyAllWindows()

# Calculate the mean IoU score
mean_iou = np.mean(iou_scores)

print(f"The mean IoU score is {mean_iou}")



image 1/1 C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Data\camera_01__2023-07-06-19-35-51-934.png: 416x640 1 traffic light, 936.4ms
Speed: 1.0ms preprocess, 936.4ms inference, 2526.3ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Data\camera_01__2023-07-06-19-35-52-034.png: 416x640 1 car, 1 traffic light, 790.0ms
Speed: 3.0ms preprocess, 790.0ms inference, 15.6ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Data\camera_01__2023-07-06-19-35-52-167.png: 416x640 1 traffic light, 815.4ms
Speed: 15.5ms preprocess, 815.4ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Dk\Projects\Team Project\YOLO detection\Test 1\Camera Data\camera_01__2023-07-06-19-35-52-234.png: 416x640 1 car, 1 traffic light, 794.6ms
Speed: 0.0ms preprocess, 794.6ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

imag

In [3]:
print(results)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant'

In [8]:
import torch
print(torch.cuda_version_1)

AttributeError: module 'torch' has no attribute 'cuda_version_1'